## training is done in this file

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from ipynb.fs.full.network import *
from ipynb.fs.full.helper import *
from ipynb.fs.full.cifar_10_parser import *

In [2]:
# if fetching data from google drive then change the address accordingly

folder_path = r'C:\Users\Admin\Desktop\program\datasets\cifar-10-batches-py'

In [3]:
# placeholder for the input images(discriminator) and noise(generator)

input_image = tf.placeholder(tf.float32, [None, 32,32,3]) # feed input images
gen_image = tf.placeholder(tf.float32, [None, 1,1,100]) # feed random noise

In [4]:
# create variables for generator and discriminator

gen = generator(gen_image)
dis_output_real, dis_logits_real = discriminator(input_image)
dis_output_fake, dis_logits_fake = discriminator(gen, reuse=True)

In [5]:
# get all the trainable variables to apply regularization

vars_g = [var for var in tf.trainable_variables() if var.name.startswith('g')]
vars_d = [var for var in tf.trainable_variables() if var.name.startswith('d')]

reg_g = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-6), vars_g)
reg_d = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-6), vars_d)

In [6]:
dis_loss_real = helper_loss(tf.ones_like(dis_logits_real), dis_logits_real)
dis_loss_fake = helper_loss(tf.zeros_like(dis_logits_fake), dis_logits_fake)

dis_loss = tf.reduce_mean((dis_loss_real + dis_loss_fake)*0.5)
gen_loss = tf.reduce_mean(helper_loss(tf.ones_like(dis_logits_fake), dis_logits_fake))

In [7]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [8]:
with tf.control_dependencies(update_ops):
    optimizer_dis = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(dis_loss + reg_d, var_list = vars_d)
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(dis_loss + reg_d, var_list = vars_d)

In [9]:
# the training takes a lot of time so create and run the session on cloud gpu

In [10]:
'''
for i in range(60000):
    train_d = True
    train_g = True
    batch_size = 50
    # keep_prob_train = 0.6 
    
    
    n = np.random.uniform(0.0, 1.0, [batch_size, 1,1,100]).astype(np.float32)   
    # batch = [np.reshape(b, [28, 28]) for b in mnist.train.next_batch(batch_size=batch_size)[0]]  
    
    d_real_ls, d_fake_ls, g_ls, d_ls = sess.run([loss_d_real, loss_d_fake, loss_g, loss_d], 
                                                feed_dict={X_in: batch, 
                                                           noise: n})
    
    d_real_ls = np.mean(d_real_ls)
    d_fake_ls = np.mean(d_fake_ls)
    g_ls = g_ls
    d_ls = d_ls
    
    if g_ls * 1.5 < d_ls:
        train_g = False
        pass
    if d_ls * 2 < g_ls:
        train_d = False
        pass
    
    if train_d:
        sess.run(optimizer_d, feed_dict={noise: n, X_in: batch})
        
        
    if train_g:
        sess.run(optimizer_g, feed_dict={gen_image: n})

'''

'\nfor i in range(60000):\n    train_d = True\n    train_g = True\n    batch_size = 50\n    # keep_prob_train = 0.6 \n    \n    \n    n = np.random.uniform(0.0, 1.0, [batch_size, 1,1,100]).astype(np.float32)   \n    # batch = [np.reshape(b, [28, 28]) for b in mnist.train.next_batch(batch_size=batch_size)[0]]  \n    \n    d_real_ls, d_fake_ls, g_ls, d_ls = sess.run([loss_d_real, loss_d_fake, loss_g, loss_d], \n                                                feed_dict={X_in: batch, \n                                                           noise: n})\n    \n    d_real_ls = np.mean(d_real_ls)\n    d_fake_ls = np.mean(d_fake_ls)\n    g_ls = g_ls\n    d_ls = d_ls\n    \n    if g_ls * 1.5 < d_ls:\n        train_g = False\n        pass\n    if d_ls * 2 < g_ls:\n        train_d = False\n        pass\n    \n    if train_d:\n        sess.run(optimizer_d, feed_dict={noise: n, X_in: batch})\n        \n        \n    if train_g:\n        sess.run(optimizer_g, feed_dict={gen_image: n})\n\n'

In [11]:
#                                                  CAUTION !!!
#                                         only run on cloud platform !!!

'''
with tf.Session() as sess:
    
    batch_size = 50
    
    num_epochs = 25
    num_batches = 200*6
    
    # input for the generator
    n = np.random.uniform(0.0, 1.0, [batch_size, 1,1,100]).astype(np.float32)
    #batch_iter = batch_generator(folder_path, batch_size)
    
    for epoch in range(num_epochs):
        
        # refresh the iterator after each epoch
        batch_iter = batch_generator(folder_path, batch_size)
        new_batch_flag = 1
        batch_counter = 0 # counts number of new batches used
        
        #for i in range(num_batches):
        while batch_counter < num_batches:
            train_d = True
            train_g = True


            # will be zero if the previous batch was not used to train the discriminator
            # so that the next batch will not be generated (the old batch will be reused)
            # problem : the loop will run 1200 times but 1200 new batches wont be used
            # possible solution : create a counter(for num of new batches used) 
            #     and use while loop inplace of the inner for loop

            if new_batch_flag == 1: 
                batch = next(batch_iter) # generate the next batch out of 200*6 batches
                batch_counter += 1

            d_real_ls, d_fake_ls, g_ls, d_ls = sess.run([dis_loss_real, dis_loss_fake, gen_loss, dis_loss], 
                                                        feed_dict={input_image: batch, gen_image: n})

            d_real_ls = np.mean(d_real_ls)
            d_fake_ls = np.mean(d_fake_ls)
            g_ls = g_ls
            d_ls = d_ls


            # the code below will create a problem
            # because the next batch will be generated but the 
            # discriminator will not be trained if train_d = False
            # so we are losing some of the training data

            # possible solution : generate a batch only "if train_d = True"
            # or remove the constraints on training
            # or create a flag as a feed back for whether or not the batch was used in the previous iteration

            if g_ls * 1.5 < d_ls:
                train_g = False
                pass
            if d_ls * 2 < g_ls:
                train_d = False
                new_batch_flag = 0
                pass
            if train_d:
                sess.run(optimizer_d, feed_dict={gen_image: n, input_image: batch})
                new_batch_flag = 1


            if train_g:
                sess.run(optimizer_g, feed_dict={gen_image: n})


'''

'\nwith tf.Session() as sess:\n    \n    batch_size = 50\n    \n    num_epochs = 25\n    num_batches = 200*6\n    \n    # input for the generator\n    n = np.random.uniform(0.0, 1.0, [batch_size, 1,1,100]).astype(np.float32)\n    #batch_iter = batch_generator(folder_path, batch_size)\n    \n    for epoch in range(num_epochs):\n        \n        # refresh the iterator after each epoch\n        batch_iter = batch_generator(folder_path, batch_size)\n        new_batch_flag = 1\n        batch_counter = 0 # counts number of new batches used\n        \n            #for i in range(num_batches):\n            while batch_counter < num_batches:\n                train_d = True\n                train_g = True\n                \n                \n                # will be zero if the previous batch was not used to train the discriminator\n                # so that the next batch will not be generated (the old batch will be reused)\n                # problem : the loop will run 1200 times but 1200 ne

In [12]:
def generate_samples():
    
    # launch a new session and input some random noise into the generator to get an image
    
    pass